# 세미프로젝트 파일정리

1. 22년 분기(1~4), 서울시 자치구, 외식 업종, 매출등급 전처리 
2. 위 데이터(키)와 개폐업, 생존율, 최신영업기간 데이터 결합 및 전처리
3. 위 데이터(키,개폐업 등)와 매출 데이터 결합 및 전처리
4. 타입변경, 문자제거 전처리
5. 탐색적 데이터 분석 및 시각화 (EDA)
6. 머신러닝 적용을 위한 주요 데이터 인코딩 전처리
7. 머신러닝 모델링 

# 1. 22년 분기(1~4), 서울시 자치구, 외식 업종, 매출등급 전처리

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
추정매출 = pd.read_csv("./서울시 상권분석서비스(상권-추정매출2022).csv", encoding = "ansi")
생활인구 = pd.read_csv("./서울시 상권분석서비스(상권-생활인구2022).csv", encoding = "ansi")
직장인구 = pd.read_csv("./서울시 상권분석서비스(상권-직장인구2022).csv", encoding = "ansi")
상권영역 = pd.read_csv("./서울시 상권분석서비스(상권영역).csv", encoding = "ansi")
법정동코드 = pd.read_csv("./서울시 건축물대장 법정동 코드정보.csv", encoding = "ansi")
매출등급 = pd.read_csv("./필지단위 소상공인 매출등급 정보.csv", encoding = "ansi")
코드변환 = pd.read_csv("./A_CS 코드변환.txt")

# 전처리 후 csv 추가 파일
년도_업종_구_등급 = pd.read_csv("./년도_업종_구_등급.csv", encoding = "ansi")

### 데이터 정의서 테이블
- "데이터기준년월", "필지고유번호", "법정동코드", "업종코드", "매출등급"

In [3]:
매출등급 = pd.read_csv("./필지단위 소상공인 매출등급 정보.csv", encoding = "ansi")

In [4]:
매출등급

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD
0,202201,1111010100100030080,11110101,A03,4
1,202201,1111010100100480000,11110101,A01,2
2,202201,1111010100100589952,11110101,A11,3
3,202201,1111010100100589952,11110101,A03,2
4,202201,1111010100100690048,11110101,A01,2
...,...,...,...,...,...
1423718,202212,1174011000106949888,11740110,A03,4
1423719,202212,1174011000106990080,11740110,A01,4
1423720,202212,1174011000107069952,11740110,A03,3
1423721,202212,1174011000107069952,11740110,A03,4


In [5]:
매출등급['INDUSTRY_CD'].value_counts()

A01    535229
A03    434959
A08     63900
A13     60933
A14     56318
A11     53078
A06     46492
A02     45419
A04     39988
A12     30267
A05     24044
A07     19004
A10      5201
A09      4463
A15      4428
Name: INDUSTRY_CD, dtype: int64

In [6]:
매출등급[매출등급['INDUSTRY_CD']=='A03']

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD
0,202201,1111010100100030080,11110101,A03,4
3,202201,1111010100100589952,11110101,A03,2
7,202201,1111010100100940032,11110101,A03,1
9,202201,1111010100100969984,11110101,A03,3
13,202201,1111010100101309952,11110101,A03,3
...,...,...,...,...,...
1423701,202212,1174011000106769920,11740110,A03,3
1423713,202212,1174011000106789888,11740110,A03,2
1423718,202212,1174011000106949888,11740110,A03,4
1423720,202212,1174011000107069952,11740110,A03,3


### 데이터 정의서에 나와있는 '업종코드' <=> 공공데이터 서울시 상권 데이터의 '서비스업종코드'를 매핑

### 데이터 정의서의 외식업종코드 15개 => 10개로 변환 
- 공공데이터인 서울시 상권 데이터에 있는 외식업종 10개 분류에 맞춤

- A01(한식),  => CS100001
- A11(중식), => CS100002
- A13(일식), => CS100003
- A14(서양식), A10(기타 외국식), => CS100004
- A04(제과제빵), A09(아이스크림/빙수), => CS100005
- A05(패스트푸드), A07(피자), => CS100006
- A02(치킨), => CS100007
- A06(분식), => CS100008
- A12(주점), => CS100009
- A08(커피), A15(음료(커피 외), => CS100010

  
- A03(기타 외식), => 제외 => 분식 CS100008 여기에 추가하면됨 (230904월)

- 대표적인 외식업체 이외 기타 외식으로 분류된 코드는 제외 함

In [7]:
코드변환 = pd.read_csv("./A_CS 코드변환.txt")

In [8]:
코드변환

,업종코드,업종,서비스업종코드
0,A01,한식,CS100001
1,A02,치킨,CS100007
2,A03,분식,CS100008
3,A04,제과제빵,CS100005
4,A05,패스트푸드,CS100006
5,A06,분식,CS100008
6,A07,피자,CS100006
7,A08,커피,CS100010
8,A09,아이스크림/빙수,CS100005
9,A10,기타 외국식,CS100004


### 컬럼 이름 한글로 변경

In [9]:
매출등급.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423723 entries, 0 to 1423722
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   DATA_CRTR_YM  1423723 non-null  int64 
 1   LT_UNQ_NO     1423723 non-null  int64 
 2   STDG_EMD_CD   1423723 non-null  int64 
 3   INDUSTRY_CD   1423723 non-null  object
 4   SLS_GRD       1423723 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 54.3+ MB


In [10]:
새로운_컬럼 = ["데이터기준년월", "필지고유번호", "법정동코드", "업종코드", "매출등급"]
매출등급.columns = 새로운_컬럼

### 사용하지 않는 필지고유번호 삭제

In [11]:
매출등급 = 매출등급.astype(str)
매출등급_1 = 매출등급.drop(columns = "필지고유번호")
매출등급_1

,데이터기준년월,법정동코드,업종코드,매출등급
0,202201,11110101,A03,4
1,202201,11110101,A01,2
2,202201,11110101,A11,3
3,202201,11110101,A03,2
4,202201,11110101,A01,2
...,...,...,...,...
1423718,202212,11740110,A03,4
1423719,202212,11740110,A01,4
1423720,202212,11740110,A03,3
1423721,202212,11740110,A03,4


### 매출등급 테이블에 업종, 서비스업종코드 칼럼 추가
- 매출등급 테이블의 업종코드에 매핑되는 업종명, 서비스업종코드 칼럼을 추가함

In [12]:
매출등급_서비스업종추가 = pd.merge(left = 매출등급_1, right = 코드변환, how = "inner")
매출등급_서비스업종추가

,데이터기준년월,법정동코드,업종코드,매출등급,업종,서비스업종코드
0,202201,11110101,A03,4,분식,CS100008
1,202201,11110101,A03,2,분식,CS100008
2,202201,11110101,A03,1,분식,CS100008
3,202201,11110101,A03,3,분식,CS100008
4,202201,11110101,A03,3,분식,CS100008
...,...,...,...,...,...,...
1423718,202212,11740108,A09,2,아이스크림/빙수,CS100005
1423719,202212,11740108,A09,4,아이스크림/빙수,CS100005
1423720,202212,11740109,A09,2,아이스크림/빙수,CS100005
1423721,202212,11740109,A09,3,아이스크림/빙수,CS100005


### 매출등급-자치구 결합
### 년월=>분기단위로 전처리
- 행정동, 법정동 전처리
- 자치구인 시군구명 칼럼 추가
- 기준년월 => 분기 단위로 전처리

In [13]:
상권영역 = pd.read_csv("./서울시 상권분석서비스(상권영역).csv", encoding = "ansi")

In [14]:
상권영역

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,202112,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,NaN
1,202112,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,NaN
2,202112,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,NaN
3,202112,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,NaN
4,202112,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,NaN
...,...,...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,동대문패션타운 관광특구,200974,451975,11140,11140590,NaN
1667,202112,U,관광특구,1001494,종로?청계 관광특구,199768,452273,11110,11110615,NaN
1668,202112,U,관광특구,1001495,잠실 관광특구,210191,446339,11710,11710566,NaN
1669,202112,U,관광특구,1001496,강남 마이스 관광특구,205310,445727,11680,11680580,NaN


In [15]:
상권영역 = 상권영역.astype(str)

In [16]:
상권영역.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   기준_년월_코드    1671 non-null   object
 1   상권_구분_코드    1671 non-null   object
 2   상권_구분_코드_명  1671 non-null   object
 3   상권_코드       1671 non-null   object
 4   상권_코드_명     1671 non-null   object
 5   엑스좌표_값      1671 non-null   object
 6   와이좌표_값      1671 non-null   object
 7   시군구_코드      1671 non-null   object
 8   행정동_코드      1671 non-null   object
 9   형태정보        1671 non-null   object
dtypes: object(10)
memory usage: 130.7+ KB


### 행정기관(행정동) 및 관할구역(법정동) 데이터를 pdr 라이브러리로 불러오기

In [17]:
import PublicDataReader as pdr

In [18]:
행_법정동코드 = pdr.code_hdong_bdong()

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 7. 3. 시행)


In [19]:
행_법정동코드

,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,법정동코드,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,1100000000,서울특별시,19880423,
1,11,서울특별시,11110,종로구,1111000000,,1111000000,종로구,19880423,
2,11,서울특별시,11110,종로구,1111000000,,1111090100,창신1동,19880423,19880423
3,11,서울특별시,11110,종로구,1111000000,,1111090200,창신2동,19880423,19880423
4,11,서울특별시,11110,종로구,1111000000,,1111090300,창신3동,19880423,19880423
...,...,...,...,...,...,...,...,...,...,...
53156,51,강원특별자치도,51830,양양군,5183035000,강현면,5183035038,장산리,20230611,
53157,51,강원특별자치도,51830,양양군,5183035000,강현면,5183035039,전진리,20230611,
53158,51,강원특별자치도,51830,양양군,5183035000,강현면,5183035040,주청리,20230611,
53159,51,강원특별자치도,51830,양양군,5183035000,강현면,5183035041,적은리,20230611,


In [20]:
행_법정동코드.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53161 entries, 0 to 53160
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도코드    53161 non-null  object
 1   시도명     53161 non-null  object
 2   시군구코드   53161 non-null  object
 3   시군구명    53161 non-null  object
 4   행정동코드   53161 non-null  object
 5   읍면동명    53161 non-null  object
 6   법정동코드   53161 non-null  object
 7   동리명     53161 non-null  object
 8   생성일자    53161 non-null  object
 9   말소일자    53161 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


In [21]:
행_법정동코드 = 행_법정동코드.loc[행_법정동코드['말소일자'] == ""]

### 법정동코드 뒤 2자리 숫자 제거 (서울 데이터에는 뒤 2자리가 필요 없음)

In [22]:
#행_법정동코드["행정동코드"] = 행_법정동코드["행정동코드"].str[:-2]
행_법정동코드["법정동코드"] = 행_법정동코드["법정동코드"].str[:-2]

In [23]:
행_법정동코드[["시군구명","법정동코드"]]

,시군구명,법정동코드
0,,11000000
1,종로구,11110000
11,종로구,11110101
12,종로구,11110102
13,종로구,11110103
...,...,...
53156,양양군,51830350
53157,양양군,51830350
53158,양양군,51830350
53159,양양군,51830350


### 법정동코드를 기준으로 중복 제거

In [24]:
행_법정동코드[["시군구명","법정동코드"]]=행_법정동코드[["시군구명","법정동코드"]].drop_duplicates(subset ="법정동코드")

In [25]:
행_법정동코드[["시군구명","법정동코드"]]

,시군구명,법정동코드
0,,11000000
1,종로구,11110000
11,종로구,11110101
12,종로구,11110102
13,종로구,11110103
...,...,...
53156,NaN,NaN
53157,NaN,NaN
53158,NaN,NaN
53159,NaN,NaN


### 매출등급 테이블에 시군구명, 법정동코드 칼럼 추가

In [26]:
df = pd.merge(left = 매출등급_서비스업종추가, right =행_법정동코드[["시군구명","법정동코드"]], how = "left")
df

,데이터기준년월,법정동코드,업종코드,매출등급,업종,서비스업종코드,시군구명
0,202201,11110101,A03,4,분식,CS100008,종로구
1,202201,11110101,A03,2,분식,CS100008,종로구
2,202201,11110101,A03,1,분식,CS100008,종로구
3,202201,11110101,A03,3,분식,CS100008,종로구
4,202201,11110101,A03,3,분식,CS100008,종로구
...,...,...,...,...,...,...,...
1423718,202212,11740108,A09,2,아이스크림/빙수,CS100005,강동구
1423719,202212,11740108,A09,4,아이스크림/빙수,CS100005,강동구
1423720,202212,11740109,A09,2,아이스크림/빙수,CS100005,강동구
1423721,202212,11740109,A09,3,아이스크림/빙수,CS100005,강동구


In [27]:
df = df.reindex(columns = ["데이터기준년월","시군구명","법정동코드","업종코드","서비스업종코드","업종","매출등급"])
df

,데이터기준년월,시군구명,법정동코드,업종코드,서비스업종코드,업종,매출등급
0,202201,종로구,11110101,A03,CS100008,분식,4
1,202201,종로구,11110101,A03,CS100008,분식,2
2,202201,종로구,11110101,A03,CS100008,분식,1
3,202201,종로구,11110101,A03,CS100008,분식,3
4,202201,종로구,11110101,A03,CS100008,분식,3
...,...,...,...,...,...,...,...
1423718,202212,강동구,11740108,A09,CS100005,아이스크림/빙수,2
1423719,202212,강동구,11740108,A09,CS100005,아이스크림/빙수,4
1423720,202212,강동구,11740109,A09,CS100005,아이스크림/빙수,2
1423721,202212,강동구,11740109,A09,CS100005,아이스크림/빙수,3


In [28]:
df2=df[['데이터기준년월', '시군구명','서비스업종코드','매출등급']]
df2

,데이터기준년월,시군구명,서비스업종코드,매출등급
0,202201,종로구,CS100008,4
1,202201,종로구,CS100008,2
2,202201,종로구,CS100008,1
3,202201,종로구,CS100008,3
4,202201,종로구,CS100008,3
...,...,...,...,...
1423718,202212,강동구,CS100005,2
1423719,202212,강동구,CS100005,4
1423720,202212,강동구,CS100005,2
1423721,202212,강동구,CS100005,3


In [29]:
df2['데이터기준년월'].value_counts()

202212    120224
202210    119934
202211    119833
202205    119708
202206    119669
202207    119643
202209    119373
202208    119177
202204    118782
202203    117138
202201    115499
202202    114743
Name: 데이터기준년월, dtype: int64

In [30]:
df2['시군구명'].value_counts()

강남구     124273
마포구      95550
송파구      82684
영등포구     75380
강서구      71420
서초구      69105
종로구      67476
관악구      60016
중구       58510
광진구      53469
강동구      52777
용산구      49175
구로구      49043
은평구      48532
동대문구     48532
성북구      47047
성동구      46446
노원구      45237
중랑구      44562
서대문구     42906
양천구      41243
동작구      41037
강북구      39949
금천구      38338
도봉구      31016
Name: 시군구명, dtype: int64

In [31]:
df2['서비스업종코드'].value_counts()

CS100001    535229
CS100008    481451
CS100010     68328
CS100004     61519
CS100003     60933
CS100002     53078
CS100007     45419
CS100005     44451
CS100006     43048
CS100009     30267
Name: 서비스업종코드, dtype: int64

In [32]:
df2['매출등급'].value_counts()

3    427062
4    355905
2    355884
1    142454
5    142418
Name: 매출등급, dtype: int64

In [33]:
df2['매출등급'] = df2['매출등급'].astype(float)

In [34]:
# df2['매출등급_로그'] = np.log10(df2['매출등급'])

In [35]:
df3=df2.groupby(['데이터기준년월','시군구명','서비스업종코드'])['매출등급'].mean().to_frame().reset_index()
df3

,데이터기준년월,시군구명,서비스업종코드,매출등급
0,202201,강남구,CS100001,2.375759
1,202201,강남구,CS100002,2.076246
2,202201,강남구,CS100003,2.406832
3,202201,강남구,CS100004,2.491826
4,202201,강남구,CS100005,2.586081
...,...,...,...,...
2995,202212,중랑구,CS100006,3.417391
2996,202212,중랑구,CS100007,3.369128
2997,202212,중랑구,CS100008,3.496308
2998,202212,중랑구,CS100009,3.369565


In [36]:
df3['매출등급'].value_counts()

3.000000    13
3.200000    11
3.250000    10
3.083333     6
3.500000     6
            ..
3.100592     1
3.216561     1
3.273743     1
3.301423     1
3.409938     1
Name: 매출등급, Length: 2673, dtype: int64

In [37]:
df3['매출등급'].describe()

count    3000.000000
mean        3.073109
std         0.323974
min         1.832579
25%         2.860997
50%         3.131902
75%         3.287914
max         3.943396
Name: 매출등급, dtype: float64

### 분기 단위로 매핑하기 위해 202203, 202206, 202209, 202212월만 추출

In [38]:
df = df[df["데이터기준년월"].str[-2:].isin(["03", "06", "09", "12"])]
df

,데이터기준년월,시군구명,법정동코드,업종코드,서비스업종코드,업종,매출등급
70060,202203,종로구,11110101,A03,CS100008,분식,4
70061,202203,종로구,11110101,A03,CS100008,분식,2
70062,202203,종로구,11110101,A03,CS100008,분식,2
70063,202203,종로구,11110101,A03,CS100008,분식,2
70064,202203,종로구,11110101,A03,CS100008,분식,3
...,...,...,...,...,...,...,...
1423718,202212,강동구,11740108,A09,CS100005,아이스크림/빙수,2
1423719,202212,강동구,11740108,A09,CS100005,아이스크림/빙수,4
1423720,202212,강동구,11740109,A09,CS100005,아이스크림/빙수,2
1423721,202212,강동구,11740109,A09,CS100005,아이스크림/빙수,3


###  '매출등급' 전처리
- 키값인 22년분기, 자치구, 외식업종 기준으로 매출등급을 평균

In [39]:
df['매출등급'] = df['매출등급'].astype(int)

In [40]:
df['매출등급'].unique()

array([4, 2, 3, 1, 5])

In [41]:
df['매출등급'].value_counts()

3    142900
4    119101
2    119088
1     47667
5     47648
Name: 매출등급, dtype: int64

In [42]:
df['매출등급']=df['매출등급'].replace({1: 5, 2: 4, 3: 3, 4: 2, 5: 1})
df['매출등급'].value_counts()

3    142900
2    119101
4    119088
5     47667
1     47648
Name: 매출등급, dtype: int64

In [43]:
df2 = df.groupby(["데이터기준년월", "서비스업종코드", "시군구명"])["매출등급"].agg(lambda x: x.mean()).to_frame().reset_index()
df2

,데이터기준년월,서비스업종코드,시군구명,매출등급
0,202203,CS100001,강남구,3.607718
1,202203,CS100001,강동구,2.775424
2,202203,CS100001,강북구,2.540466
3,202203,CS100001,강서구,3.015158
4,202203,CS100001,관악구,2.721897
...,...,...,...,...
995,202212,CS100010,용산구,3.025641
996,202212,CS100010,은평구,2.659218
997,202212,CS100010,종로구,3.205776
998,202212,CS100010,중구,3.551020


### '매출등급' 조정

In [44]:
df2['매출등급'].describe()

count    1000.000000
mean        2.927808
std         0.326662
min         2.056604
25%         2.713943
50%         2.863203
75%         3.142857
max         4.126531
Name: 매출등급, dtype: float64

In [45]:
df2.loc[df2['매출등급'] >= 2.863203 , '매출등급01'] = 1
df2.loc[df2['매출등급'] < 2.863203 , '매출등급01'] = 0

In [46]:
df2['매출등급01'].value_counts()

1.0    500
0.0    500
Name: 매출등급01, dtype: int64

In [47]:
import pandas as pd

# DataFrame에서 '매출등급01' 열의 값 카운트
value_counts = df2['매출등급01'].value_counts()

# 0과 1의 값 비율 계산
ratio_0 = value_counts[0] / len(df2['매출등급01'])
ratio_1 = value_counts[1] / len(df2['매출등급01'])

# 결과 출력
print("0의 비율:", ratio_0)
print("1의 비율:", ratio_1)

0의 비율: 0.5
1의 비율: 0.5


In [48]:
df2['매출등급01']

0      1.0
1      0.0
2      0.0
3      1.0
4      0.0
      ... 
995    1.0
996    0.0
997    1.0
998    1.0
999    0.0
Name: 매출등급01, Length: 1000, dtype: float64

In [49]:
df2['매출등급01']=df2['매출등급01'].astype(np.int64)

In [50]:
df2

,데이터기준년월,서비스업종코드,시군구명,매출등급,매출등급01
0,202203,CS100001,강남구,3.607718,1
1,202203,CS100001,강동구,2.775424,0
2,202203,CS100001,강북구,2.540466,0
3,202203,CS100001,강서구,3.015158,1
4,202203,CS100001,관악구,2.721897,0
...,...,...,...,...,...
995,202212,CS100010,용산구,3.025641,1
996,202212,CS100010,은평구,2.659218,0
997,202212,CS100010,종로구,3.205776,1
998,202212,CS100010,중구,3.551020,1


### 업종명 추가

In [51]:
c = 코드변환.drop_duplicates(subset = "서비스업종코드")

In [52]:
c

,업종코드,업종,서비스업종코드
0,A01,한식,CS100001
1,A02,치킨,CS100007
2,A03,분식,CS100008
3,A04,제과제빵,CS100005
4,A05,패스트푸드,CS100006
7,A08,커피,CS100010
9,A10,기타 외국식,CS100004
10,A11,중식,CS100002
11,A12,주점,CS100009
12,A13,일식,CS100003


- CS100001 => 한식
- ACS100002 => 중식
- CS100003 => 일식
- CS100004 => 기타 외국식(서양식 포함)
- CS100005 => 제과제빵(아이스크림/빙수 포함)
- CS100006 => 패스트푸드(피자 포함)
- CS100007 => 치킨
- CS100008 => 분식
- CS100009 => 주점
- CS100010 => 커피(음료(커피 외) 포함)

In [53]:
df2["서비스업종"] = df2["서비스업종코드"].map(dict(zip(c["서비스업종코드"],c["업종"])))

In [54]:
df2

,데이터기준년월,서비스업종코드,시군구명,매출등급,매출등급01,서비스업종
0,202203,CS100001,강남구,3.607718,1,한식
1,202203,CS100001,강동구,2.775424,0,한식
2,202203,CS100001,강북구,2.540466,0,한식
3,202203,CS100001,강서구,3.015158,1,한식
4,202203,CS100001,관악구,2.721897,0,한식
...,...,...,...,...,...,...
995,202212,CS100010,용산구,3.025641,1,커피
996,202212,CS100010,은평구,2.659218,0,커피
997,202212,CS100010,종로구,3.205776,1,커피
998,202212,CS100010,중구,3.551020,1,커피


### CSV파일 저장

In [55]:
df2.to_csv("./년도_업종_구_등급.csv", encoding='ansi')